In [ ]:
import json
import pandas
import pyarrow

In [ ]:
pandas.set_option('display.max_colwidth', None)

In [ ]:
# Chemin vers le fichier JSONL
filename = '/home/onyxia/work/openfoodfacts-products.jsonl'

In [ ]:
# Fonction pour afficher de manière jolie les premières lignes du fichier
def pretty_print_first_lines(filename, num_lines=3):
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            if i >= num_lines:
                break

            # Charger la ligne JSON
            data = json.loads(line)

            # Afficher le dictionnaire formaté
            print(json.dumps(data, indent=4))

In [ ]:
# Afficher les 3 premières lignes avec la fonction précédente
pretty_print_first_lines(filename)

In [ ]:
# Fonction pour extraire les lignes correspondant à une valeur de code donnée
def extract_lines_with_code(filename, code_value):
    
    lines_with_code = []

    with open(filename, 'r') as file:
        for line in file:
            # Charger la ligne JSON
            data = json.loads(line)

            # Vérifier si le champ "code" a la valeur spécifiée
            if data.get('code') == code_value:
                lines_with_code.append(data)

    return lines_with_code

In [ ]:
# Valeur du champ "code" à rechercher avec la fonction précédente
code_value = "3272770005708"

# Extraction des lignes avec le champ "code" égal à la valeur spécifiée
lines_with_code = extract_lines_with_code(
    filename, 
    code_value
)

# Conversion en DataFrame pandas
df = pandas.DataFrame(lines_with_code)

# Affichage des premières lignes du DataFrame
print(head(df))

In [ ]:
# Fonction pour extraire les lignes correspondant à une valeur de code donnée sachant que code est unique dans le fichier
def extract_lines_with_code_optimisee(filename, code_value):
    with open(filename, 'r') as file:
        for line in file:
            # Charger la ligne JSON
            data = json.loads(line)

            # Vérifier si le champ "code" a la valeur spécifiée
            if data.get('code') == code_value:
                return data  # Retourner la ligne trouvée
            elif data.get('code') > code_value:
                break  # Arrêter la recherche si le code actuel dépasse la valeur recherchée

    return None  # Retourner None si aucune ligne avec le code spécifié n'est trouvée

In [ ]:
# Valeur du champ "code" à rechercher
code_value = "3272770005708"

# Récupération de la ligne avec le champ "code" égal à la valeur spécifiée
line_with_code2 = extract_lines_with_code_optimisee(
    filename, 
    code_value
)

# Conversion en DataFrame pandas
df2 = pandas.DataFrame(lines_with_code2)

# Affichage des premières lignes du DataFrame
print(head(df2))

In [ ]:
# Fonction pour extraire les lignes dont le prefixe du champ code commence par un valeur donnée
# et ne retenir que les variables code et ingredients
def extract_lines_with_code_starting_with(filename, code_prefix):
    lines_with_code = []

    with open(filename, 'r') as file:
        for line in file:
            # Charger la ligne JSON
            data = json.loads(line)

            # Vérifier si le champ "code" commence par la valeur spécifiée
            if data.get('code', '').startswith(code_prefix):
                # Filtrer et conserver uniquement les champs "code" et "ingredients"
                filtered_data = {'code': code, 'ingredients': data.get('ingredients', [])}
                lines_with_code.append(filtered_data)

    return lines_with_code

In [ ]:
# Préfixe du champ "code" à rechercher
code_prefix = "3"

# Extraction des lignes avec le champ "code" commençant par le préfixe spécifié
lines_with_code3 = extract_lines_with_code_starting_with(
    filename, 
    code_prefix
)

# Conversion en DataFrame pandas
df3 = pandas.DataFrame(lines_with_code3)

# Affichage des premières lignes du DataFrame
print(head(df3))

In [ ]:
# Fonction pour extraire les lignes dont le prefixe du champ code commence par une liste de valeurs données
# et ne retenir que les variables code et ingredients
def filter_lines_with_code_prefix(filename, code_prefixes):
    lines_with_code = []

    with open(filename, 'r') as file:
        for line in file:
            # Charger la ligne JSON
            data = json.loads(line)

            # Récupérer le champ "code"
            code = data.get('code', '')

            # Vérifier si le champ "code" commence par l'un des préfixes spécifiés
            if any(code.startswith(prefix) for prefix in code_prefixes):
                # Filtrer et conserver uniquement les champs "code" et "ingredients"
                filtered_data = {'code': code, 'ingredients': data.get('ingredients', [])}
                lines_with_code.append(filtered_data)

    return lines_with_code

In [ ]:
# Préfixes des codes à filtrer
code_prefixes = ['30', '31', '32', '33', '34', '35', '36', '37']

# Filtrage des lignes avec les préfixes de code spécifiés
lines_with_code_prefix4 = filter_lines_with_code_prefix(
    filename, 
    code_prefixes
)

# Conversion en DataFrame pandas
df4 = pandas.DataFrame(lines_with_code_prefix4)

In [ ]:
# Affichage des premières lignes du DataFrame et récupération d'informations sur le DataFrame
print(df4.head(10))
df4.count()
df4.dtypes

In [ ]:
# Export en local du conteneur dans un fichier csv
df4.to_csv("/home/onyxia/work/data_interet.csv", index=False)

In [ ]:
# Première tentative d'un xport en local du conteneur dans un fichier parquet
df4.to_parquet("/home/onyxia/work/data_interet.parquet", index=False, dtype={'ingredients': str})
    # erreur lié à certaines valeurs de la variable ingredients

In [ ]:
# Transformation adéquate pour résoudre l'erreur précédente
df4["ingredients_transform"] = df4['ingredients'].astype(str)
df4.drop(columns=['ingredients'], inplace=True)

In [ ]:
# Export en local du conteneur dans un fichier parquet
df4.to_parquet("/home/onyxia/work/data_interet.parquet", index=False, dtype={'ingredients': str})